This notebook merges the player information in the matches files and the point by point data in the points files. Only fields that will be needed in the extraction notebook(s) will be written to the resulting csv. For now that means the 2 player names, the point server, the point winner and the score of the respective players after the point. More can be added in the future if needed (add to end of f_fields list!). The surface of the tournament is also added as a field.

Slightly modified to get set5 experiment data (Changed first block & return_path)

In [271]:
# information to change per use
year = "2011"
points_file = year+"-wimbledon-points.csv"
matches_file = year+"-wimbledon-matches.csv"
return_file = year+"-wimbledon-pbp-no-set5.csv"
surface = "hard" # valid options: "grass","hard","clay"
f_fields = ["P1","P2","Surface","match_id","PointServer","PointWinner","P1Score","P2Score","SetNo"] # match_id is the key

In [272]:
import os
import csv

In [273]:
path = os.getcwd() # "...\\Code" parallel with "Data Lake"
path = path[:-4]+"Data Lake\\Point-By-Point\\" # all involved files use this same path
points_path = path+points_file
matches_path = path+matches_file
# return_path = path+return_file
return_path = path+"set5\\"+return_file

In [274]:
points = [] # each item to be a dict containing info on a single point

# go through points data and keep specified fields
with open(points_path,'r') as f:
    
    csvreader = csv.reader(f)
    fields = next(csvreader)
    
    for row in csvreader:
        # initialize point dict
        # only need first two as rest initialized upon entry below
        
        # block below inserted later to extract 5th set specific points
        # will be commented out after use
        # if row[fields.index("SetNo")]=='5':
        #     continue
        
        point = {}
        point["P1"] = ""
        point["P2"] = ""
        point["Surface"] = surface
        for field in f_fields[3:]:
            point[field] = row[fields.index(field)]
        points.append(point)

In [275]:
# now using match_id fill in "P1","P2"
with open(matches_path,'r') as f:
    
    csvreader = csv.reader(f)
    fields = next(csvreader)
    match_id = fields.index("match_id")
    P1 = fields.index("player1")
    P2 = fields.index("player2")
    i = 0
    for row in csvreader:
        try:
            while row[match_id]==points[i]["match_id"]:
                points[i]["P1"] = row[P1]
                points[i]["P2"] = row[P2]
                i += 1
        except IndexError:
            if i==len(points):
                pass
            else:
                print("Indexing Error")

In [276]:
# no need for match_id anymore
f_fields.remove("match_id")
for point in points:
    point.pop("match_id")

# writing points list
with open(return_path,'w',newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=f_fields)
    writer.writeheader()
    writer.writerows(points)